In [3]:
import numpy as np
import pandas as pd

train_path = "X_train.csv"
label_path = "y_train.csv"
test_path = "X_test.csv"
samples_path = "samples.csv"

df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
labeldf = pd.read_csv(label_path)


labels = labeldf.values
data = df.values
test_data = test_df.values

data = data[:,1:]
test_data = test_data[:,1:]
labels = labels[:,1]


print(data.shape)
print(test_data.shape)
print(labels.shape)

(5117, 17813)
(3411, 17813)
(5117,)


In [4]:
def count_instances(labels):
    unique, counts = np.unique(labels,return_counts=True)
    return dict(zip(unique,counts))

class_counts = count_instances(labels)

print("Class 0 Counts: {}".format(class_counts[0]))
print("Class 1 Counts: {}".format(class_counts[1]))
print("Class 2 Counts: {}".format(class_counts[2]))
print("Class 3 Counts: {}".format(class_counts[3]))


(num_train, max_timesteps) = data.shape
(num_test, _) = test_data.shape

Class 0 Counts: 3030
Class 1 Counts: 443
Class 2 Counts: 1474
Class 3 Counts: 170


In [5]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(1)

In [6]:
# # truncate and pad input sequences
max_review_length = data.shape[1]
# X_train = sequence.pad_sequences(data, maxlen=max_review_length, padding='post')
# X_test = sequence.pad_sequences(test_data, maxlen=max_review_length, padding='post')
X_train = np.nan_to_num(data, nan = 0)
X_test = np.nan_to_num(test_data,nan=0)

In [7]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [8]:
X_train.shape

(5117, 1, 17813)

In [ ]:
import keras
labels = keras.utils.to_categorical(labels)

In [16]:
from sklearn.utils import class_weight
y_train = labeldf.values[:,1]
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
        
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [21]:
# create and fit the LSTM network
number_of_classes = 4
batch_size = 32
model = Sequential()
model.add(LSTM(256, return_sequences=True, input_shape=(1, 17813)))
model.add(Dropout(0.25))
# model.add(LSTM(256, return_sequences=True))
# model.add(Dropout(0.25))
# model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(0.25))
# model.add(LSTM(64, return_sequences=True))
# model.add(Dropout(0.25))
model.add(LSTM(32))
model.add(Dense(number_of_classes, activation='softmax'))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', f1_m])
model.fit(X_train, labels, epochs=250, batch_size=batch_size, validation_split=0.2, verbose=2, shuffle=False, class_weight=class_weights, callbacks=[early_stopping])
# model.save('Keras_models/my_model_' + str(i) + '_' + str(j) + '_' + str() + '.h5')
# predictions = model.predict(X_val)
# score = accuracy_score(change(Y_val), change(predictions))
# print(score)

Train on 4093 samples, validate on 1024 samples
Epoch 1/250
 - 8s - loss: 1.2664 - accuracy: 0.5121 - val_loss: 1.1546 - val_accuracy: 0.5762
Epoch 2/250
 - 7s - loss: 1.0583 - accuracy: 0.5959 - val_loss: 1.0308 - val_accuracy: 0.5801
Epoch 3/250
 - 7s - loss: 0.9982 - accuracy: 0.5947 - val_loss: 1.0153 - val_accuracy: 0.5801
Epoch 4/250
 - 7s - loss: 0.9839 - accuracy: 0.5952 - val_loss: 1.0278 - val_accuracy: 0.5781
Epoch 5/250
 - 7s - loss: 0.9810 - accuracy: 0.5947 - val_loss: 1.0221 - val_accuracy: 0.5771
Epoch 6/250
 - 7s - loss: 0.9752 - accuracy: 0.5947 - val_loss: 1.0245 - val_accuracy: 0.5791
Epoch 7/250
 - 7s - loss: 0.9723 - accuracy: 0.5959 - val_loss: 1.0205 - val_accuracy: 0.5781
Epoch 8/250
 - 7s - loss: 0.9692 - accuracy: 0.5976 - val_loss: 1.0193 - val_accuracy: 0.5771
Epoch 9/250
 - 7s - loss: 0.9678 - accuracy: 0.5966 - val_loss: 1.0164 - val_accuracy: 0.5811
Epoch 10/250
 - 7s - loss: 0.9666 - accuracy: 0.5971 - val_loss: 1.0111 - val_accuracy: 0.5830
Epoch 11/25